In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import sys
import time
import argparse
import pdb
import numpy as np
import pandas as pd
from sklearn.metrics import normalized_mutual_info_score, accuracy_score

#Keras import
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Merge, Embedding, merge, Activation, Reshape
from keras import backend as K
from keras.regularizers import l2



Using Theano backend.


In [3]:
edges = pd.read_csv("data/dblp_cleaned_edges.txt", sep="\t")
edges.columns = ["node1", "node2"]

In [4]:
node_coms = pd.read_csv("data/dblp_cleaned_com.txt", sep="\t")
node_coms.columns = ["node", "com"]

In [5]:
n_nodes = node_coms.shape[0]
latent_dim = 50
n_coms = pd.unique(node_coms.com).size

In [9]:
input1 = Input(shape=(1,), dtype="int32")
input2 = Input(shape=(1,), dtype="int32")

embed_layer = Embedding(n_nodes, latent_dim, W_regularizer=l2(0.001))
x1 = embed_layer(input1)
x2 = embed_layer(input2)

reshape_layer = Reshape((latent_dim,), input_shape=(1, latent_dim))
hidden_layer = Dense(latent_dim, activation="linear", W_regularizer=l2(0.001))
x1 = reshape_layer(x1)
x2 = reshape_layer(x2)
x1 = hidden_layer(x1)
x2 = hidden_layer(x2)

x_merged = merge([x1, x2], mode="mul")

prob = Dense(1, activation="sigmoid")(x_merged)

model = Model([input1, input2], prob)

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [7]:
train_s = edges["node1"]
train_o = edges["node2"]
labels = np.ones(train_s.size)

In [11]:
neg_ratio = 0.5
n_pos = labels.size
n_neg = int(n_pos * neg_ratio)
rand1 = np.random.randint(0, n_nodes, n_neg)
rand2 = np.random.randint(0, n_nodes, n_neg)

seq = np.arange(n_pos + n_neg)
np.random.shuffle(seq)
concat1 = np.append(train_s, rand1)[seq]
concat2 = np.append(train_o, rand2)[seq]
labels = np.append(np.ones(n_pos), np.zeros(n_neg))[seq]

In [8]:
model.fit([train_s, train_o], labels)

Epoch 1/10
 24512/202230 [==>...........................] - ETA: 389s - loss: 0.1215 - acc: 0.9980

KeyboardInterrupt: 